In [13]:
import xarray as xr
import rioxarray

regrowth = rioxarray.open_rasterio(f'/Users/gclyne/Downloads/CA_forest_harvest_years2recovery/CA_forest_harvest_years2recovery.tif',decode_coords='all',lock=False,chunks=True).sel(band=1)
# regrowth.rio.reproject('EPSG:4326').rio.to_raster(f'/Users/gclyne/Downloads/CA_forest_harvest_years2recovery/CA_forest_harvest_years2recovery_reprojected.tif')


In [ ]:
regrowth.rio.crs

In [ ]:
regrowth_sub = regrowth.sel(x=slice(-2660895.524,-2560895.524),y=slice(2998833.1105,2898833.1105))

In [ ]:
import numpy as np
np.unique(x.data,return_counts=True)

In [8]:
import pandas as pd
ecozones_coords = pd.read_csv('/Users/gclyne/thesis/data/ecozones_coordinates.csv')
from preprocessing.utils import getGeometryBoxes

boxes = getGeometryBoxes(ecozones_coords)

In [14]:
import pyproj
from shapely.ops import transform


wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:3978')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
transformed_box = transform(project, boxes[0])
bbox = transformed_box.bounds

In [15]:
s = regrowth.sel(x=slice(bbox[0],bbox[2]),y=slice(bbox[3],bbox[1]))


In [16]:
np.unique(s,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], dtype=int8),
 array([17816419,      959,      820,      480,      348,      222,
             206,      180,      171,      100,       69,       64,
              50,       29,       48,       44,       29,       30,
              48,       37,       23,       31,       21,       23,
              24,        7,       10,        4,        3,        7,
               4]))

In [21]:
452/17819004

2.536617647091835e-05

In [17]:
959/17816419    

5.3826753849917876e-05

In [18]:
228/6153236

3.7053673871764385e-05

In [ ]:
s.rio.to_raster('sel_raster.tif')

In [ ]:
geometries = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [425499.18381405267, 4615331.540546387],
            [425499.18381405267, 4615478.540546387],
            [425526.18381405267, 4615478.540546387],
            [425526.18381405267, 4615331.540546387],
            [425499.18381405267, 4615331.540546387]
        ]]
    }
]    
lon = bbox[0]
next_lon = bbox[2]
lat = bbox[3]
next_lat = bbox[1]
from shapely.geometry import Polygon
poly = Polygon([(lon,next_lat),(lon,lat),(next_lon,lat),(next_lon,next_lat)])


In [ ]:
clipped = regrowth.rio.clip([poly],crs='EPSG:3978',drop=True)

In [ ]:
clipped

In [ ]:
clipped.rio.to_raster('clipped_raster.tif')

In [ ]:
import numpy as np
np.unique(clipped,return_counts=True)

In [ ]:
boxes[0].bounds

In [12]:
from preprocessing.utils import clipNFIS

regrowth = rioxarray.open_rasterio(f'/Users/gclyne/Downloads/CA_forest_harvest_years2recovery/CA_forest_harvest_years2recovery.tif',decode_coords='all',lock=False,chunks=True)
clipped = clipNFIS(regrowth,boxes[0].bounds[1],boxes[0].bounds[0],boxes[0].bounds[3],boxes[0].bounds[2])

In [20]:
np.unique(clipped,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30], dtype=int8),
 array([17819004,      452,      331,      250,       93,       54,
              58,       51,       43,       18,       19,       18,
              15,        6,       13,        8,        8,       13,
              10,       11,        4,        8,        4,        3,
               3,        3,        5,        2,        3]))

In [ ]:
#convert numpy unqiue counts to dataframe
def convertUniquesToDF(uniq):
    df = pd.DataFrame(uniq[1],index=uniq[0])
    df = df.reset_index()
    df.columns = ['value','count']
    return df


In [ ]:
import pyproj
from shapely.ops import transform


wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:3978')


out_df = pd.DataFrame()
for i in boxes:
    clipped = clipNFIS(regrowth,i.bounds[1],i.bounds[0],i.bounds[3],i.bounds[2])
    print(i.bounds)
    project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
    transformed_box = transform(project, i)
    bbox = transformed_box.bounds
    c = clipped.sel(x=slice(bbox[0],bbox[2]),y=slice(bbox[3],bbox[1]))
    uniq = np.unique(c,return_counts=True)
    df = convertUniquesToDF(uniq)
    df['lat'] = i.bounds[1]
    df['lon'] = i.bounds[0]
    #append two dataframes
    def append_df(df1, df2):
        return pd.concat([df1,df2], ignore_index=True)
    out_df = append_df(df,out_df)


In [10]:
import rioxarray 
import numpy as np
regrowth_4326 = rioxarray.open_rasterio(f'/Users/gclyne/thesis/output_raster.tif',decode_coords='all',lock=False,chunks=True)

for i in boxes:
    out = regrowth_4326.sel(x=slice(i.bounds[0],i.bounds[2]),y=slice(i.bounds[3],i.bounds[1]))
    print(np.unique(out,return_counts=True))
    break



(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 27], dtype=uint8), array([6153236,     228,     150,     131,      47,      26,      31,
            28,      26,      12,       9,      10,       9,       4,
             4,       1,       2,       5,       1,       6,       4,
             2,       1,       2,       1,       1,       1]))


In [11]:
out.rio.to_raster('out_raster.tif')

In [47]:
pd.read_csv('data/cesm_data_variant.csv')

,year,lat,lon,cOther,cCwd,cVeg,cSoilAbove1m,cLitter,cLeaf,cRoot,...,wetlandFrac,ps,pr,tas_DJF,tas_JJA,tas_MAM,tas_SON,grassCropFrac,variant,area
0,1850.0,41.937172,-82.50,0.051695,0.075508,0.382676,1.049892,0.028913,0.006591,0.081992,...,3.118757,98932.187500,0.000028,271.092743,296.431274,282.682648,286.598175,5.525701,1.0,1.077156e+10
1,1850.0,42.879582,-81.25,0.365441,0.924869,3.454496,7.368393,0.224013,0.039276,0.747683,...,5.387547,98765.304688,0.000028,269.867340,295.002899,281.763916,284.794617,29.421182,1.0,1.061060e+10
2,1850.0,42.879582,-80.00,0.248429,0.564599,2.127686,5.366026,0.168735,0.035755,0.465783,...,0.218983,98405.867188,0.000030,269.754913,294.573395,281.042877,284.825470,24.893268,1.0,1.061060e+10
3,1850.0,43.821991,-81.25,0.412050,1.170400,3.918015,9.144390,0.254259,0.040559,0.847211,...,0.208533,98645.570312,0.000026,268.902679,293.489899,280.233887,283.826385,34.626941,1.0,1.044666e+10
4,1850.0,43.821991,-80.00,0.398501,0.877372,2.943997,10.256255,0.273317,0.055604,0.643991,...,0.000000,98548.742188,0.000027,268.864777,293.191925,280.160583,283.308472,53.209544,1.0,1.044666e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028175,2014.0,83.403145,-73.75,0.000000,0.000000,0.000000,0.244392,0.005502,0.000000,0.000000,...,0.000000,98576.882812,0.000007,242.772339,271.818695,252.222290,258.384857,2.765289,7.0,1.567847e+09
2028176,2014.0,83.403145,-72.50,0.000000,0.000000,0.000000,0.146004,0.000307,0.000000,0.000000,...,0.000000,98596.843750,0.000007,242.643265,271.813354,252.203690,258.152008,0.119568,7.0,1.567847e+09
2028177,2014.0,83.403145,-71.25,0.000000,0.000000,0.000000,0.145932,0.000000,0.000000,0.000000,...,0.000000,98673.312500,0.000007,242.460007,271.718292,252.090530,257.986755,0.000000,7.0,1.567847e+09
2028178,2014.0,83.403145,-70.00,0.000000,0.000000,0.000000,23.031897,0.043866,0.000000,0.000000,...,0.000000,98804.664062,0.000007,242.284775,271.662750,252.004227,257.858551,18.550827,7.0,1.567847e+09


In [6]:
import rioxarray 
import pandas as pd
import numpy as np
ecozones_coords = pd.read_csv('/Users/gclyne/thesis/data/ecozones_coordinates.csv')
ecozone_coords = ecozones_coords[ecozones_coords['zone'].isin(['Boreal Cordillera','Boreal PLain', 'Boreal Shield'])]

from preprocessing.utils import getGeometryBoxes
boxes = getGeometryBoxes(ecozones_coords)
forest_df = pd.DataFrame()
for_har = rioxarray.open_rasterio('data/reprojected_4326_CA_Forest_Harvest_1985-2020_test.tif',chunks=True)
for year in range(1985,1986):
    print(year)
    cur_forest = rioxarray.open_rasterio(f'home/gclyne/scratch/reprojected_4326_CA_forest_{year}.tif',chunks=True)
    last_year_forest = rioxarray.open_rasterio(f'home/gclyne/scratch/reprojected_4326_CA_forest_{year}.tif',chunks=True)
    for box in boxes:
        print(box.bounds)
        lon = box.bounds[0]
        next_lon = box.bounds[2]
        lat = box.bounds[1]
        next_lat = box.bounds[3]
        cur_cell = cur_forest.sel(band=1,x=slice(lon,next_lon),y=slice(next_lat,lat))
        last_year_cell = last_year_forest.sel(band=1,x=slice(lon,next_lon),y=slice(next_lat,lat))
        for_har_cell = for_har.sel(band=1,x=slice(lon,next_lon),y=slice(next_lat,lat))
        cur_cell = np.where(cur_cell.data > 200,1,0)
        last_year_cell = np.where(last_year_cell.data > 200,1,0)
        for_har_prev_harvest = np.where(for_har_cell.data < year,1,0)
        harvested = np.where(for_har_cell.data == year,1,0)
        #needs to be previously harvested, and not considered forest in the previous year
        new_growth = np.where((cur_cell == 1) & (last_year_cell == 0) & (for_har_prev_harvest == 1),1,0)
        total_size = np.size(cur_cell)
        percentage_growth = np.count_nonzero(new_growth) / total_size
        tree_cover = np.count_nonzero(cur_cell) / total_size
        percent_harvested = np.count_nonzero(harvested) / total_size

        forest_df = pd.concat([forest_df,{'lat':lat,'lon':lon,'year':year,'percentage_growth':percentage_growth,'tree_cover':tree_cover,'percent_harvested':percent_harvested}],ignore_index=True)
        break
# forest_df.to_csv('forest_df.csv')

1985
(-65.0, 57.015705, -63.75, 57.95811462402344)


/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/var/folders/1g/_wddvqdx1zn1sqjjw3s9m2940000gt/T/ipykernel_18440/23

(-65.0, 57.958115, -63.75, 58.9005241394043)


/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/Users/gclyne/thesis/env/lib/python3.9/site-packages/dask/array/core.py:1712: FutureWarning: The `numpy.size` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/var/folders/1g/_wddvqdx1zn1sqjjw3s9m2940000gt/T/ipykernel_18440/23

KeyboardInterrupt: 